In [2]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.data_structures.ids_classifier import mine_CARs
from pyids.model_selection import mode
import time
import os

from pyarc.qcba.transformation import QCBATransformation

from pyarc import CBA
from pyarc.data_structures import TransactionDB
from pyarc.qcba.data_structures import QuantitativeDataFrame
import pandas as pd

from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

from pyarc.qcba.classifier import QuantitativeClassifier
from sklearn.metrics import accuracy_score

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

QuantitativeCAR.interval_reader = interval_reader

basepath="C:/code/python/machine_learning/"
unique_transactions= True

In [3]:
def run1fold(basepath, datasetname, unique_transactions=True, runQCBA=False, saveIDSRules=True):
    df_stat = pd.DataFrame(columns=['ids', 'idsqcba'], index=["accuracy", "rulecount", "rulelength", "buildtime"])

    if (runQCBA):        
        #python QCBA implementation uses custom discretization format
        data_train_disc = pd.read_csv(basepath+"data/folds_discr/train/{}.csv".format(datasetname))        
        data_test_disc = pd.read_csv(basepath+"data/folds_discr/test/{}.csv".format(datasetname))
        data_test_undisc = pd.read_csv(basepath+"data/folds/test/{}.csv".format(datasetname))
        data_train_undisc = pd.read_csv(basepath+"data/folds/train/{}.csv".format(datasetname))
        
        quant_dataframe_test_undisc = QuantitativeDataFrame(data_test_undisc)
        quant_dataframe_train_undisc = QuantitativeDataFrame(data_train_undisc)
    else:
        #R QCBA implementation uses different discretization format
        data_train_disc = pd.read_csv(basepath+"data/folds_discr/train/{}.csv".format(datasetname))        
        data_test_disc = pd.read_csv(basepath+"data/folds_discr/test/{}.csv".format(datasetname))

    
    
    quant_dataframe_train_disc = QuantitativeDataFrame(data_train_disc)
    quant_dataframe_test_disc = QuantitativeDataFrame(data_test_disc)
    
    actual = quant_dataframe_test_disc.dataframe.iloc[:,-1].values
    
    #learn rules for IDS
    cars = mine_CARs(data_train_disc, 50, sample=True)
    #train IDS model
    ids = IDS()
    start = time.time()
    ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train_disc, debug=False)
    end = time.time()
    df_stat.loc["buildtime","ids"] = end-start
    #apply IDS model
    # ?? bylo quant_dataframe_train_disc
    df_stat.loc["accuracy","ids"] =  ids.score(quant_dataframe_test_disc)    
    print("Acc IDS:",df_stat.loc["accuracy","ids"]  )
    df_stat.loc["rulecount","ids"] = len(ids.clf.rules)    
    antLengths=list(map(lambda r: len(r.car.antecedent.itemset.items()),ids.clf.rules ))
    df_stat.loc["rulelength","ids"]=sum(antLengths)/len(antLengths)
    
    avg_rule_legnth_ids = None
    print("Rule Count IDS:", df_stat.loc["rulecount","ids"])
    if (saveIDSRules):
        idsRulesPath = basepath+"idsModels/{}.csv".format(datasetname)
        file = open(idsRulesPath,"w")
        txtexport="rules,suppport,confidence,lift\n"
        for r in ids.clf.rules:
            args = [r.car.antecedent.string(), "{" + r.car.consequent.string() + "}", r.car.support, r.car.confidence,0]
            txtexport = txtexport+ "\"{} => {}\",{:.2f},{:.2f},{:.2f} \n".format(*args) 
        #add default rule
        classname = data_train_disc.columns.values[-1]
        txtexport = txtexport+ "\"{ } => " + "{"+classname+"="+ mode(data_train_disc[data_train_disc.columns[-1]])+ "}\", 0,0,0"

        print(txtexport)    
            
        file.write(txtexport)
        file.close()

    if (runQCBA):
        #postprocess IDS model with QCBA
        rules_to_optimize = ids.clf.rules
        start = time.time()
        quant_rules = [ QuantitativeCAR(r.car) for r in rules_to_optimize ]
        qcba_transformation = QCBATransformation(quant_dataframe_train_undisc)
        transformed_rules = qcba_transformation.transform(quant_rules)
        end = time.time()
        df_stat.loc["buildtime","idsqcba"] = end-start
        rules, default_class = transformed_rules    
        antLengths=list(map(lambda r: len(r.car.antecedent.itemset.items()),ids.clf.rules ))
        #+1 because the default rule is not counted
        df_stat.loc["rulelength","idsqcba"]=sum(antLengths)/(len(antLengths)+1)

        #apply QCBA model        
        qclf = QuantitativeClassifier(rules, default_class)
        # ?? Lze předat undisc data? bylo quant_dataframe_train_disc
        pred = qclf.predict(quant_dataframe_test_undisc) 
        
        #evaluate model - QCBA
        df_stat.loc["accuracy","idsqcba"] =  accuracy_score(actual, pred)    
        df_stat.loc["rulecount","idsqcba"] = len(rules)
        print("Acc IDS-QCBA:", df_stat.loc["accuracy","idsqcba"] )
        print("Rule Count IDS-QCBA:", df_stat.loc["rulecount","idsqcba"])
    return df_stat

def mean_allfolds(dataset_name, start=0, end=10, unique_transactions=True,runQCBA=False):
    files = [ dataset_name + repr(i) for i in range(start, end) ]
    df_agg = None
    emptyDF = True
    for file in files:
        df_stats = run1fold(basepath,file, unique_transactions=unique_transactions,runQCBA=runQCBA)
        print("done", file) 
        if emptyDF:
            df_agg = df_stats
            emptyDF = False
        else:
            df_agg = df_agg + df_stats     
        print(df_agg)
    foldcount = end-start
    print(foldcount)
    print(df_agg)
    df_agg= df_agg / foldcount    
    print(df_agg)
    return df_agg


In [ ]:
datasets = ["australian","anneal","autos","breast-w","colic","credit-a","credit-g","diabetes","glass","heart-statlog","ionosphere","iris","labor","letter","lymph","segment","sonar","spambase","vehicle","vowel","hepatitis","hypothyroid"]    

resultFolder="ids"
runQCBA = False
resultfilesuffix=""
if not(runQCBA):
    resultfilesuffix="-ids"
if not os.path.exists(resultFolder):
    os.makedirs(resultFolder)
for dataset in datasets:
    print(dataset)
    resultFile=resultFolder+"/"+dataset+resultfilesuffix+"-result.csv"
    if os.path.exists(resultFile):
        print("skipping already computed")
        continue
    df_stats_per_dataset = mean_allfolds(dataset,runQCBA)    
    df_stats_per_dataset.to_csv(resultFile)
    print("*****")
    print(df_stats_per_dataset)

australian
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=15
Rule count: 577, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=15
Rule count: 4101, Iteration: 2
Target rule count satisfied: 1000
cover cache prepared
overlap cache prepared
Acc IDS: 0.6
Rule Count IDS: 20
rules,suppport,confidence,lift
"{X3=-inf_to_-0.6994645,X6=-inf_to_0.625,X2=All} => {Y=-1}",0.35,0.71,0.00 
"{X14=-0.9921_to_inf,X8=0.0_to_inf,X4=-0.5_to_inf} => {Y=1}",0.17,0.97,0.00 
"{X7=-0.9284215_to_inf,X1=All,X5=-inf_to_0.153846} => {Y=1}",0.14,0.52,0.00 
"{X14=-inf_to_-0.9921,X3=-0.6994645_to_inf,X11=All} => {Y=1}",0.13,0.49,0.00 
"{X10=-inf_to_-0.9253735,X7=-0.9284215_to_inf,X6=-inf_to_0.625} => {Y=-1}",0.12,0.60,0.00 
"{X10=-0.9253735_to_inf,X5=0.153846_to_inf,X8=0.0_to_inf} => {Y=1}",0.11,0.95,0.00 
"{X3=-0.6994645_to_inf,X6=-inf_to_0.625} => {Y=-1}",0.13,0.44,0.00 
"{X3=-inf_to_-0.6994645,X4

cover cache prepared
overlap cache prepared
Acc IDS: 0.463768115942029
Rule Count IDS: 20
rules,suppport,confidence,lift
"{X12=All,X13=-0.9005_to_inf,X11=All} => {Y=1}",0.26,0.37,0.00 
"{X5=0.4615385_to_inf,X8=0.0_to_inf,X4=-0.5_to_inf} => {Y=1}",0.14,0.90,0.00 
"{X13=-0.9005_to_inf,X4=-inf_to_-0.5,X10=-inf_to_-0.9850745} => {Y=-1}",0.12,0.90,0.00 
"{X5=-0.153846_to_0.4615385,X8=0.0_to_inf,X9=0.0_to_inf} => {Y=1}",0.12,0.94,0.00 
"{X3=-0.6994645_to_inf,X13=-inf_to_-0.9005,X9=0.0_to_inf} => {Y=1}",0.10,0.88,0.00 
"{X7=-inf_to_-0.9284215} => {Y=1}",0.14,0.26,0.00 
"{X6=0.625_to_inf,X2=All} => {Y=-1}",0.07,0.36,0.00 
"{X13=-0.9005_to_inf,X7=-inf_to_-0.9284215,X1=All} => {Y=1}",0.09,0.22,0.00 
"{X7=-0.9284215_to_inf,X11=All,X4=-inf_to_-0.5} => {Y=-1}",0.06,0.67,0.00 
"{X6=-inf_to_0.625,X3=-0.6994645_to_inf,X5=-inf_to_-0.153846} => {Y=1}",0.06,0.45,0.00 
"{X12=All,X9=0.0_to_inf,X5=-inf_to_-0.153846} => {Y=-1}",0.05,0.45,0.00 
"{X5=-0.153846_to_0.4615385,X9=0.0_to_inf,X4=-0.5_to_inf} => {Y=-

overlap cache prepared
Acc IDS: 0.6617647058823529
Rule Count IDS: 19
rules,suppport,confidence,lift
"{X11=All,X10=-inf_to_-0.9850745,X9=-inf_to_0.0} => {Y=-1}",0.44,0.76,0.00 
"{X4=All,X5=-inf_to_0.153846} => {Y=-1}",0.43,0.67,0.00 
"{X13=-0.9005_to_inf,X1=All,X5=-inf_to_0.153846} => {Y=-1}",0.32,0.73,0.00 
"{X10=-0.9253735_to_inf,X12=All,X6=-0.875_to_inf} => {Y=1}",0.22,0.90,0.00 
"{X11=All,X3=-0.700893_to_inf,X9=0.0_to_inf} => {Y=1}",0.17,0.82,0.00 
"{X5=0.153846_to_inf,X11=All,X14=-0.99491_to_inf} => {Y=1}",0.12,0.87,0.00 
"{X6=-0.875_to_inf,X14=-inf_to_-0.99491,X9=0.0_to_inf} => {Y=1}",0.12,0.61,0.00 
"{X10=-0.9253735_to_inf,X7=-0.9284215_to_inf,X5=0.153846_to_inf} => {Y=1}",0.09,0.93,0.00 
"{X10=-0.9850745_to_-0.9253735,X6=-0.875_to_inf} => {Y=1}",0.08,0.53,0.00 
"{X6=-inf_to_-0.875,X11=All,X5=-inf_to_0.153846} => {Y=-1}",0.07,0.88,0.00 
"{X10=-0.9850745_to_-0.9253735,X14=-inf_to_-0.99491,X2=All} => {Y=-1}",0.07,0.65,0.00 
"{X14=-0.99491_to_inf,X7=-inf_to_-0.9284215,X9=0.0_to_inf

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=16
Rule count: 2658, Iteration: 1
Target rule count satisfied: 1000
cover cache prepared
overlap cache prepared
Acc IDS: 0.5824175824175825
Rule Count IDS: 17
rules,suppport,confidence,lift
"{shape=COIL,packing=All} => {class=3}",0.37,0.81,0.00 
"{enamelability=-inf_to_1.82142857143,surface-quality=G} => {class=3}",0.22,0.84,0.00 
"{len=-inf_to_0.5,formability=2.10741444867_to_2.60741444867} => {class=3}",0.19,0.83,0.00 
"{formability=2.60741444867_to_inf,product-type=C} => {class=3}",0.09,0.51,0.00 
"{product-type=C,family=ZS} => {class=3}",0.05,0.78,0.00 
"{surface-quality=E,thick=2.1_to_inf} => {class=3}",0.05,0.97,0.00 
"{condition=S,width=-inf_to_1167.5} => {class=2}",0.05,0.13,0.00 
"{len=-inf_to_0.5,packing=All} => {class=5}",0.03,0.07,0.00 
"{width=1235.0_to_1410.05,strength=-inf_to_375.0} => {class=2}",0.02,0.09,0.00 
"{thick=0.7995_to_0.9005,enamelability=-inf_to_1.82142857143} => {cla

cover cache prepared
overlap cache prepared
Acc IDS: 0.38636363636363635
Rule Count IDS: 16
rules,suppport,confidence,lift
"{steel=R,hardness=-inf_to_22.5} => {class=2}",0.08,0.27,0.00 
"{width=-inf_to_1167.5,cbond=Y} => {class=3}",0.06,0.94,0.00 
"{shape=SHEET,surface-quality=D} => {class=3}",0.04,1.00,0.00 
"{non-ageing=N,len=0.5_to_inf} => {class=5}",0.03,0.55,0.00 
"{product-type=C,surface-quality=G} => {class=U}",0.03,0.11,0.00 
"{shape=SHEET,product-type=C} => {class=U}",0.02,0.04,0.00 
"{shape=SHEET,packing=All} => {class=U}",0.02,0.04,0.00 
"{formability=2.60789980732_to_inf,width=1167.5_to_1235.0} => {class=5}",0.01,0.83,0.00 
"{thick=1.5495_to_1.5995,carbon=-inf_to_3.5} => {class=3}",0.01,0.34,0.00 
"{formability=2.60789980732_to_inf,bw/me=B} => {class=5}",0.01,0.43,0.00 
"{len=-inf_to_0.5,thick=0.7995_to_0.8005} => {class=2}",0.01,0.16,0.00 
"{bore=All,cbond=Y} => {class=U}",0.01,0.07,0.00 
"{carbon=-inf_to_3.5,blue/bright/varn/clean=B} => {class=2}",0.00,1.00,0.00 
"{formab

...

cover cache prepared
overlap cache prepared
Acc IDS: 0.5
Rule Count IDS: 16
rules,suppport,confidence,lift
"{statutory-holidays=All,wage-increase-second-year=2.75_to_inf,contribution-to-health-plan=full} => {class=good}",0.25,0.93,0.00 
"{wage-increase-first-year=-inf_to_2.9,statutory-holidays=All,duration=All} => {class=bad}",0.24,0.92,0.00 
"{wage-increase-third-year=All,duration=All,standby-pay=All} => {class=bad}",0.35,0.35,0.00 
"{contribution-to-dental-plan=half,wage-increase-second-year=2.75_to_inf,duration=All} => {class=bad}",0.08,0.33,0.00 
"{shift-differential=All,education-allowance=no,contribution-to-dental-plan=none} => {class=bad}",0.06,1.00,0.00 
"{wage-increase-second-year=2.75_to_inf,education-allowance=no,standby-pay=All} => {class=good}",0.06,0.33,0.00 
"{contribution-to-dental-plan=half,contribution-to-health-plan=none,standby-pay=All} => {class=bad}",0.04,1.00,0.00 
"{wage-increase-third-year=All,pension=none,longterm-disability-assistance=yes} => {class=bad}",0.0

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=17
Rule count: 84505, Iteration: 1
Target rule count satisfied: 1000
cover cache prepared
overlap cache prepared
Acc IDS: 0.037257824143070044
Rule Count IDS: 15
rules,suppport,confidence,lift
"{x-bar=6.5_to_7.5,xy2br=7.5_to_8.5} => {class=O}",0.01,0.07,0.00 
"{xybar=5.5_to_6.5} => {class=B}",0.01,0.06,0.00 
"{width=0.5_to_3.5,high=-inf_to_8.5} => {class=Z}",0.01,0.04,0.00 
"{xy2br=7.5_to_8.5,y2bar=5.5_to_6.5} => {class=J}",0.00,0.07,0.00 
"{y-ege=-inf_to_0.5,x2ybr=2.5_to_4.5} => {class=H}",0.00,0.18,0.00 
"{y-ege=3.5_to_4.5,onpix=5.5_to_8.5} => {class=D}",0.00,0.07,0.00 
"{x-bar=6.5_to_7.5,x2ybr=4.5_to_5.5} => {class=M}",0.00,0.04,0.00 
"{x2ybr=5.5_to_6.5,onpix=5.5_to_8.5} => {class=V}",0.00,0.02,0.00 
"{yegvx=4.5_to_5.5,x2bar=3.5_to_4.5} => {class=Y}",0.00,0.08,0.00 
"{high=8.5_to_9.5} => {class=K}",0.00,0.02,0.00 
"{y2bar=-inf_to_0.5,xybar=6.5_to_7.5} => {class=K}",0.00,0.03,0.00 
"{y2bar=3.5

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=17
Rule count: 84162, Iteration: 1
Target rule count satisfied: 1000
cover cache prepared
overlap cache prepared
Acc IDS: 0.06118355065195587
Rule Count IDS: 18
rules,suppport,confidence,lift
"{onpix=-inf_to_1.5,yegvx=7.5_to_8.5} => {class=Y}",0.01,0.10,0.00 
"{xegvy=7.5_to_8.5,y-bar=7.5_to_8.5} => {class=E}",0.01,0.11,0.00 
"{onpix=5.5_to_9.5,xybar=6.5_to_7.5} => {class=H}",0.00,0.06,0.00 
"{onpix=5.5_to_9.5,xybar=5.5_to_6.5} => {class=G}",0.00,0.08,0.00 
"{x-box=1.5_to_2.5,x2ybr=8.5_to_9.5} => {class=U}",0.00,0.17,0.00 
"{xybar=10.5_to_11.5,x2ybr=4.5_to_5.5} => {class=P}",0.00,0.16,0.00 
"{high=8.5_to_9.5,xybar=5.5_to_6.5} => {class=Q}",0.00,0.39,0.00 
"{yegvx=7.5_to_8.5,xybar=4.5_to_5.5} => {class=V}",0.00,0.07,0.00 
"{y-bar=10.5_to_11.5,y2bar=5.5_to_6.5} => {class=Y}",0.00,0.16,0.00 
"{x-ege=1.5_to_2.5,y-ege=4.5_to_5.5} => {class=T}",0.00,0.03,0.00 
"{x2ybr=5.5_to_6.5,width=9.5_to_inf} => {c

cover cache prepared
overlap cache prepared
Acc IDS: 0.5714285714285714
Rule Count IDS: 18
rules,suppport,confidence,lift
"{regeneration_of=no,block_of_affere=yes} => {class=metastases}",0.44,0.83,0.00 
"{defect_in_node=lac_margin,changes_in_node=lac_margin} => {class=metastases}",0.23,0.86,0.00 
"{exclusion_of_no=yes,by_pass=no} => {class=metastases}",0.26,0.47,0.00 
"{lym_nodes_dimin=-inf_to_1.5,defect_in_node=lac_central} => {class=malign_lymph}",0.19,0.56,0.00 
"{dislocation_of=yes,extravasates=yes} => {class=metastases}",0.18,0.50,0.00 
"{extravasates=yes,no_of_nodes_in=-inf_to_3.5} => {class=malign_lymph}",0.10,0.27,0.00 
"{extravasates=yes,changes_in_node=lacunar} => {class=malign_lymph}",0.07,0.59,0.00 
"{lym_nodes_enlar=2.5_to_inf,by_pass=no} => {class=metastases}",0.07,0.24,0.00 
"{lymphatics=deformed,extravasates=no} => {class=malign_lymph}",0.04,0.38,0.00 
"{changes_in_lym=oval,changes_in_stru=coarse} => {class=malign_lymph}",0.04,0.38,0.00 
"{changes_in_stru=drop_like,lymp

cover cache prepared
overlap cache prepared
Acc IDS: 0.42857142857142855
Rule Count IDS: 11
rules,suppport,confidence,lift
"{lym_nodes_enlar=-inf_to_2.5,by_pass=no} => {class=metastases}",0.31,0.68,0.00 
"{exclusion_of_no=no,bl_of_lymph_c=no} => {class=metastases}",0.13,0.68,0.00 
"{changes_in_node=lac_margin,defect_in_node=lacunar} => {class=metastases}",0.10,0.88,0.00 
"{changes_in_stru=faint,bl_of_lymph_c=no} => {class=metastases}",0.10,0.38,0.00 
"{by_pass=yes,exclusion_of_no=yes} => {class=malign_lymph}",0.07,0.33,0.00 
"{lym_nodes_enlar=-inf_to_2.5,dislocation_of=no} => {class=malign_lymph}",0.06,0.22,0.00 
"{lymphatics=deformed,changes_in_node=lacunar} => {class=malign_lymph}",0.04,0.50,0.00 
"{by_pass=yes} => {class=fibrosis}",0.03,0.12,0.00 
"{block_of_affere=no,defect_in_node=no} => {class=normal}",0.01,0.67,0.00 
"{bl_of_lymph_c=yes,changes_in_node=lacunar} => {class=malign_lymph}",0.01,0.33,0.00 
"{changes_in_stru=diluted,changes_in_node=lac_central} => {class=metastases}",